(RM05)=
# RM05

## Linear Programming

Linear Programming (LP; same abbreviation for "Linear Program") is
a linear optimization technique
which became widely known
for **resource planning tasks** after World War II.
As the name suggests,
the objective function and (in-)equality constraint functions are linear.

> The term "programming" should be understood as "planning" or "conception",
> rather than the action of writing a program in a particular programming language.

In this work the following **LP standard form** is chosen:

$$
\begin{array}{lll}
\textrm{minimize}   & c^{T} x \\
\textrm{subject to} & Ax = b \\
& x \geq 0,
\end{array}
$$

where $c,x \in \mathbb{R}^{n}$, $A \in \mathbb{R}^{m \times n}$, and $b \in \mathbb{R}^{m}$.

## Meeting other LP standard forms

To compute an optimal LP solution,
there exist many functions or programs,
which are referred to in short as **solver**.
An LP solver usually specifies an own standard form.
In the following a few **conversion techniques** are described.

- "minimize $c^{T}x$" is equivalent to "maximize $-c^{T}x$".
  **Flip the sign** of the computed objective value.
- **Slack variables** $s \in \mathbb{R}^{m}$
  help expressing inequality constraints as equality constraints:
  
  $$
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x \\
  \textrm{subject to} & Ax {\color{red}{\;\leq\;}} b \\
  & x \geq 0
  \end{array}
  \quad\iff\quad
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x {\color{red}{\;+\; [0]^{T} s}} \\
  \textrm{subject to} & Ax      {\color{red}{\;+\; Is =}}\; b \\
  & x \geq 0 \\
  & {\color{red}{s \geq 0}}
  \end{array}
  $$
  
  or
  
  $$
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x \\
  \textrm{subject to} & Ax {\color{red}{\;\geq\;}} b \\
  & x \geq 0
  \end{array}
  \quad\iff\quad
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x {\color{red}{\;+\; [0]^{T} s}} \\
  \textrm{subject to} & Ax      {\color{red}{\;-\; Is =}}\; b \\
  & x \geq 0 \\
  & {\color{red}{s \geq 0}}
  \end{array}
  $$
  
  where $I$ is the $m \times m$ unit matrix (main diagonal elements are $1$, $0$ otherwise)
  and $[0] \in \mathbb{R}^{m}$ is a vector of zeros.

- **Free variables** $y \in \mathbb{R}^{k}$ can be modeled
  as difference of two non-negative LP variables $y := y^{+} - y^{-}$:

  $$
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x {\color{red}{\;+\;d^{T} y}} \\
  \textrm{subject to} & Ax {\color{red}{\;+\;By}} = b \\
  & x \geq 0 \\
  & {\color{red}{y \quad\text{"free"}}}
  \end{array}
  \quad\iff\quad
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x {\color{red}{\;+\;d^{T} y^{+} \;-\; d^{T} y^{-}}} \\
  \textrm{subject to} & Ax {\color{red}{\;+\;By^{+} \;-\; By^{-}}} = b \\
  & x \geq 0 \\
  & {\color{red}{y^{+} \geq 0}} \\
  & {\color{red}{y^{-} \geq 0}}
  \end{array}
  $$
  
  However,
  due to the introduced redundancy this approach might fail in practical applications
  and other techniques are known, see {cite}`Luenberger2016` (part I).

- Expressing **box constraints** $l,x,u \in \mathbb{R}^{n}$.

  $$
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x \\
  \textrm{subject to} & Ax = b \\
  & {\color{red}{l \;\leq\; x \;\leq\; u}}
  \end{array}
  \quad\iff\quad
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x \\
  \textrm{subject to} & Ax = b \\
  & {\color{red}{-Ix\;\leq\;-l}} \\
  & {\color{red}{Ix\;\leq\;u}} \\
  & {\color{red}{x \quad\text{"free"}}}
  \end{array}
  $$
  
  where $I$ is the $n \times n$ unit matrix (main diagonal elements are $1$, $0$ otherwise).
  See also the note about "free variables" above.

- Expressing equality constraints with inequality constraints.

  $$
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x \\
  \textrm{subject to} & Ax {\color{red}{\;=\;}} b \\
  & x \geq 0
  \end{array}
  \quad\iff\quad
  \begin{array}{ll}
  \textrm{minimize}   & c^{T} x \\
  \textrm{subject to} & Ax {\color{red}{\;\leq\;}} b\; {\color{red}{(\;+\;\varepsilon)}} \\
  & Ax {\color{red}{\;\geq\;}} b\; {\color{red}{(\;-\;\varepsilon)}} \\
  & x \geq 0
  \end{array}
  $$
  
  The resulting problem is usually ill-posed
  and a small tolerance $\varepsilon > 0$ can be added to each element of $b$ for numerical stability.

For more information on LP and equivalent formulations,
see {cite}`Luenberger2016` (part I) and {cite}`Boyd2004` (chapter 4).

## Example: Car factory

![convex_sets](res/quantamagazine_lp.png)

Source: [quantamagazine (2021-11-01)](https://www.quantamagazine.org/surprising-limits-discovered-in-quest-for-optimal-solutions-20211101/).

In [1]:
c = -[ 10000 20000 ];  % Maximize profit margin
A = [ 30 100; ...
      30  30 ];
b = [ 20000; 8000 ];
Aeq = []; % No equality constraints
beq = [];
lb = [   0   0 ];
ub = [ inf inf ];
CTYPE = repmat ('U', 2, 1);  % Octave: A(i,:)*x <= b(i)
x0 = [];  % default start value
%[x,~,exitflag] = linprog(c,A,b,Aeq,beq,lb,ub,x0);  % Matlab: exitflag=1 success
[x,fval,exitflag] = glpk(c,A,b,lb,ub,CTYPE)         % Octave: exitflag=0 success

x =

    95.238
   171.429

fval = -4.3810e+06
exitflag = 0


This result is of course very "technical" and needs interpretation to be useful for the car company.

The quantity of produced cars should be integers.
A safe strategy is rounding the results down:

In [2]:
cheapo = floor (x(1))
deluxe = floor (x(2))

cheapo = 95
deluxe = 171


The computed profit `fval` is negative.
However to adapt to the standard form of a minimization problem,
the objective function was negated.

Furthermore,
after rounding the values, the new profit can be computed from the objective function:

In [3]:
profit = -c * [cheapo; deluxe]  % dollar

profit = 4370000


Finally,
the budgets for the painting and assembly line time can be checked with `A` and `b`:

In [4]:
required_time = A * [cheapo; deluxe]  % minutes

required_time =

   19950
    7980



In [5]:
permitted_time = b  % minutes

permitted_time =

   20000
    8000



Thus the problem is optimally solved.
Not a single deluxe or cheapo could be manufactured with the given time budget:

In [6]:
remaining_budget = permitted_time - required_time  % minutes

remaining_budget =

   50
   20

